In [ ]:
import graphviz as gv
from IPython.display import Image, display
from flask import Flask, render_template, request
import csv

productos = []
with open('Short_Products.txt', 'r', encoding='utf-8') as archivo:
    lector = csv.DictReader(archivo)
    for fila in lector:
        productos.append({
            "nombre": fila['product'],
            "category": fila['category'],
            "rating": float(fila['rating'])
        })

def sugerir_productos_dfs(producto_nombre, num_recomendaciones=5):
    producto_ingresado = next((p for p in productos if producto_nombre.lower() in p['nombre'].lower()), None)

    if not producto_ingresado:
        return f"Producto '{producto_nombre}' no encontrado."

    grafo = {}
    for p in productos:
        if p['category'] not in grafo:
            grafo[p['category']] = []
        grafo[p['category']].append(p)

    stack = grafo[producto_ingresado['category']]
    visitados = set()
    productos_relacionados = []

    while stack:
        producto = stack.pop()
        if producto['nombre'] not in visitados:
            visitados.add(producto['nombre'])
            if producto['category'] == producto_ingresado['category'] and producto['nombre'] != producto_ingresado['nombre']:
                productos_relacionados.append(producto)

    productos_relacionados.sort(key=lambda x: x['rating'], reverse=True)

    recomendaciones = productos_relacionados[:num_recomendaciones]

    if recomendaciones:
        return recomendaciones
    else:
        return "No se encontraron productos relacionados."

def show(productos, directed=False, max_edges=300000, layout="dot"):
    g = gv.Digraph("direccionado") if directed else gv.Graph()

    g.graph_attr["layout"] = layout
    g.edge_attr["color"] = "gray"
    g.node_attr["color"] = "orangered"
    g.node_attr["width"] = "0.1"
    g.node_attr["height"] = "0.1"
    g.node_attr["fontsize"] = "8"
    g.node_attr["fontcolor"] = "mediumslateblue"
    g.node_attr["fontname"] = "monospace"
    g.edge_attr["fontcolor"] = "mediumslateblue"
    g.edge_attr["fontname"] = "monospace"

    for i, producto in enumerate(productos):
        g.node(str(i), label=f"{producto['nombre']}\nRating: {producto['rating']}")

    added = set()
    aristas_agregadas = 0
    for i in range(len(productos)):
        for j in range(i + 1, len(productos)):
            if aristas_agregadas >= max_edges:
                break
            if productos[i]["category"] == productos[j]["category"]:
                if directed:
                    g.edge(str(i), str(j), label=f"Rating: {productos[j]['rating']}")
                else:
                    if not f"{i}, {j}" in added:
                        g.edge(str(i), str(j), label=f"Rating: {productos[j]['rating']}")
                        added.add(f"{i}, {j}")
                        added.add(f"{j}, {i}")
                aristas_agregadas += 1
    return g

In [ ]:
grafo = show(productos, directed=False, max_edges=300000, layout="sfdp")

grafo.render('grafo_productos', format='png')

display(Image(filename='grafo_productos.png'))

sugerencia = sugerir_productos_dfs('dove')

print(sugerencia)